https://tensorflow.google.cn/guide/eager

https://github.com/tensorflow/docs/blob/master/site/en/tutorials/eager/eager_basics.ipynb

TensorFlow 的 Eager Execution 是一种命令式编程环境，可立即评估操作，无需构建图：操作会立即返回具体的值，而不是构建以后再运行的计算图。  

Eager Execution 一旦启用，便无法在同一程序中停用。

## 设置和基本用法
要启动 Eager Execution，请将 tf.enable_eager_execution() 添加到程序或控制台会话的开头。不要将此操作添加到程序调用的其他模块。

In [1]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf

In [2]:
tf.executing_eagerly()

False

In [3]:
print(tf.add(1, 2))

Tensor("Add:0", shape=(), dtype=int32)


In [4]:
tf.enable_eager_execution()
tf.executing_eagerly()

ValueError: tf.enable_eager_execution must be called at program startup.

In [2]:
tf.enable_eager_execution()
tf.executing_eagerly()

True

In [3]:
print(tf.add(1, 2))

tf.Tensor(3, shape=(), dtype=int32)


## Tensors
A Tensor is a multi-dimensional array. Similar to NumPy ndarray objects,

In [2]:
print(tf.add([1, 2], [3, 4]))
print(tf.square(5))
print(tf.reduce_sum([1, 2, 3]))
print(tf.encode_base64("hello world"))

# Operator overloading is also supported
print(tf.square(2) + tf.square(3))

tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor([4 6], shape=(2,), dtype=int32)
tf.Tensor(25, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(b'aGVsbG8gd29ybGQ', shape=(), dtype=string)
tf.Tensor(13, shape=(), dtype=int32)


Each Tensor has a shape and a datatype

In [3]:
x = tf.matmul([[1]], [[2, 3]])
print(x.shape)
print(x.dtype)

(1, 2)
<dtype: 'int32'>


#### The most obvious differences between NumPy arrays and TensorFlow Tensors are:

* Tensors can be backed by accelerator memory (like GPU, TPU).
* Tensors are immutable.


Eager Execution 适合与 NumPy 一起使用。NumPy 操作接受 tf.Tensor 参数。TensorFlow 数学运算将 Python 对象和 NumPy 数组转换为 tf.Tensor 对象。tf.Tensor.numpy 方法返回对象的值作为 NumPy ndarray。

## NumPy Compatibility
Conversion between TensorFlow Tensors and NumPy ndarrays is quite simple as:

* TensorFlow operations automatically convert NumPy ndarrays to Tensors.
* NumPy operations automatically convert Tensors to NumPy ndarrays.


Tensors can be explicitly converted to NumPy ndarrays by invoking the .numpy() method on them. These conversions are typically cheap as the array and Tensor share the underlying memory representation if possible. However, sharing the underlying representation isn't always possible since the Tensor may be hosted in GPU memory while NumPy arrays are always backed by host memory, and the conversion will thus involve a copy from GPU to host memory.

In [4]:
import numpy as np

ndarray = np.ones([3, 3])

print("TensorFlow operations convert numpy arrays to Tensors automatically")
tensor = tf.multiply(ndarray, 42)
print(tensor)


print("And NumPy operations convert Tensors to numpy arrays automatically")
print(np.add(tensor, 1))

print("The .numpy() method explicitly converts a Tensor to a numpy array")
print(tensor.numpy())

TensorFlow operations convert numpy arrays to Tensors automatically
tf.Tensor(
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]], shape=(3, 3), dtype=float64)
And NumPy operations convert Tensors to numpy arrays automatically
[[43. 43. 43.]
 [43. 43. 43.]
 [43. 43. 43.]]
The .numpy() method explicitly converts a Tensor to a numpy array
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]]


In [5]:
x = tf.random_uniform([3, 3])

print("Is there a GPU available: "),
print(tf.test.is_gpu_available())

print("Is the Tensor on GPU #0:  "),
print(x.device.endswith('GPU:0'))

Is there a GPU available: 
False
Is the Tensor on GPU #0:  
False


## Datasets
This section demonstrates the use of the tf.data.Dataset API to build pipelines to feed data to your model. It covers:

* Creating a Dataset.
* Iteration over a Dataset with eager execution enabled.


In [6]:
ds_tensors = tf.data.Dataset.from_tensor_slices([1, 2, 3, 4, 5, 6])

In [10]:
ds_tensors

<DatasetV1Adapter shapes: (), types: tf.int32>

In [7]:
# Create a CSV file
import tempfile
_, filename = tempfile.mkstemp()

In [8]:
filename

'/var/folders/7j/kgtjln3x2dj2g2v57d5vncyw0000gp/T/tmppfre7oxy'

In [9]:
with open(filename, 'w') as f:
  f.write("""Line 1
Line 2
Line 3
  """)

ds_file = tf.data.TextLineDataset(filename)

In [11]:
ds_tensors = ds_tensors.map(tf.square).shuffle(2).batch(2)

ds_file = ds_file.batch(2)

## Iterate
When eager execution is enabled Dataset objects support iteration. 

In [12]:
print('Elements of ds_tensors:')
for x in ds_tensors:
  print(x)

print('\nElements in ds_file:')
for x in ds_file:
  print(x)

Elements of ds_tensors:
Instructions for updating:
Colocations handled automatically by placer.
tf.Tensor([1 4], shape=(2,), dtype=int32)
tf.Tensor([16  9], shape=(2,), dtype=int32)
tf.Tensor([25 36], shape=(2,), dtype=int32)

Elements in ds_file:
tf.Tensor([b'Line 1' b'Line 2'], shape=(2,), dtype=string)
tf.Tensor([b'Line 3' b'  '], shape=(2,), dtype=string)
